In [7]:
import yaml
import numpy as np
from scipy.spatial.transform import Rotation as R

def rotmat_from_quat(q):
    q = np.array(q)
    q = q/np.linalg.norm(q)
    return R.from_quat(q).as_matrix()

def calc_actuator_mixing_matrix(config_file):
    with open(config_file, 'r') as stream:
        try:
            config = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)

    n_motors = len(config)
    actuator_mixing_matrix = np.zeros((6,n_motors))
    for i in range(n_motors):
        vec_z_mf = np.array([0,0,1]) # motor frame z axis
        q_i = np.array(config['motor_'+str(i)]['orientation'])
        # rotation matrix from quaternion [x,y,z,w]
        rotmat_i = rotmat_from_quat(q_i)
        t_i = np.array(config['motor_'+str(i)]['translation'])
        torque_direction = -config['motor_'+str(i)]['direction']
        torque_constant = config['motor_'+str(i)]['torque_constant']
        vec_z_bf = rotmat_i @ vec_z_mf # motor axis in body frame
        # forces
        actuator_mixing_matrix[:3,i] = vec_z_bf
        # torques
        actuator_mixing_matrix[3:,i] = np.cross(t_i, vec_z_bf) + vec_z_bf*torque_constant*torque_direction
    return actuator_mixing_matrix

In [8]:
filename = 'motor_config.yaml'
mixing_matrix = calc_actuator_mixing_matrix(filename)
print(mixing_matrix)


[[ 0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.    ]
 [ 1.      1.      1.      1.      1.      1.    ]
 [-0.0785 -0.0785  0.0785  0.0785 -0.0785  0.0785]
 [-0.0785  0.0785  0.0785 -0.0785  0.      0.    ]
 [-0.01   -0.01    0.01    0.01    0.01   -0.01  ]]


['motor_0', 'motor_1', 'motor_2', 'motor_3', 'motor_4', 'motor_5']


In [23]:
import urdfpy

filename = "/home/mihir/workspaces/aerial_gym_ws/aerial_gym_simulator/resources/robots/morphy/morphy.urdf"
urdf = urdfpy.URDF.load(filename)
print(urdf)
n_motors = 6
motor_names = ['motor_'+str(i) for i in range(n_motors)]
print(motor_names)

# get transformation of motor link wrt base link
motor_transforms = {}
links_fk = urdf.link_fk()
for key, value in links_fk.items():
    if key.name in motor_names:
        transform = value
        print(key.name, transform[:, 3])
        r, p, y = urdfpy.matrix_to_rpy(transform)
        print(key.name, r, p, y)


['motor_0', 'motor_1', 'motor_2', 'motor_3', 'motor_4', 'motor_5']
motor_0 [ 8.94974747e-02 -8.14974747e-02  3.42761220e-13  1.00000000e+00]
motor_0 0.0 -0.0 -0.78539816339
motor_1 [-8.94974747e-02 -8.14974747e-02  3.42761220e-13  1.00000000e+00]
motor_1 0.0 -0.0 -2.35619449019
motor_2 [-8.94974747e-02  8.14974747e-02  3.42761220e-13  1.00000000e+00]
motor_2 0.0 -0.0 2.3561944901923453
motor_3 [8.94974747e-02 8.14974747e-02 3.42761220e-13 1.00000000e+00]
motor_3 0.0 -0.0 0.7853981633974483
